In [ ]:
!pip install gradio transformers torch

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
model_name = "VityaVitalich/LLaMA3-8b-CQA"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
if torch.cuda.is_available():
    model = model.to('cuda')

In [ ]:
prompt = "Cats or dogs?"

inputs = tokenizer(prompt, return_tensors="pt")

if torch.cuda.is_available():
    inputs = inputs.to('cuda')

output = model.generate(**inputs, max_length=500, num_return_sequences=1)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
# def generate_text(prompt):
#     inputs = tokenizer(prompt, return_tensors="pt")

#     if torch.cuda.is_available():
#         inputs = inputs.to('cuda')

#     output = model.generate(**inputs, max_length=500, num_return_sequences=1)
#     return tokenizer.decode(output[0], skip_special_tokens=True)

# gr.Interface(fn=generate_text, inputs="example: Cats or dogs?", outputs="text", title="CQA").launch(share=True)

In [ ]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = inputs.to('cuda')
    output = model.generate(**inputs, max_length=500, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

with gr.Blocks(
    theme=gr.themes.Monochrome(),
    title="CQA",
    css="""
        .input-text {border-radius: 8px; border: 1px solid #ddd;}
        .output-text {background-color: #f9f9f9; border-radius: 8px;}
        button {border-radius: 6px;}
    """
) as demo:
    gr.Markdown(
        """
        # CQA demo
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            input_text = gr.Textbox(
                label="Your Question",
                placeholder="Example: Cats or dogs?",
                lines=2,
                max_lines=5,
                elem_classes="input-text"
            )
            submit_btn = gr.Button("Generate Response", variant="primary")

        with gr.Column(scale=2):
            output_text = gr.Textbox(
                label="Comparison Summary",
                lines=10,
                max_lines=20,
                interactive=False,
                elem_classes="output-text"
            )

    gr.Markdown(
        """
        ### CQA demo
        This interface uses LLaMA3-8b-CQA model.
        """
    )

    submit_btn.click(
        fn=generate_text,
        inputs=input_text,
        outputs=output_text
    )

demo.launch(share=True, inline=True)

In [ ]:
#!gradio deploy